Text Embedding - IMDB dataset
=============================
---
Introduction au Deep Learning  (IDLE) - S. Arias, E. Maldonado, JL. Parouty - CNRS/SARI/DEVLOG - 2020  

## Reviews analysis :

The objective is to guess whether our new and personals films reviews are **positive or negative** .  
For this, we will use our previously saved model.

What we're going to do:

 - Preparing the data
 - Retrieve our saved model
 - Evaluate the result


## Step 1 - Init python stuff

In [1]:
import numpy as np

import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.datasets.imdb as imdb

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import pandas as pd

import os,h5py,json,re

import fidle.pwk as ooo
from importlib import reload

ooo.init()

IDLE 2020 - Practical Work Module
  Version            : 0.2.4
  Run time           : Monday 27 January 2020, 21:57:44
  Matplotlib style   : fidle/talk.mplstyle
  TensorFlow version : 2.0.0
  Keras version      : 2.2.4-tf


## Step 2 : Preparing the data
### 2.1 - Our reviews :

In [79]:
reviews = [ "This film is particularly nice, a must see.",
             "Some films are classics and cannot be ignored.",
             "This movie is just abominable and doesn't deserve to be seen!"]

### 2.2 - Retrieve dictionaries

In [80]:
with open('./data/word_index.json', 'r') as fp:
    word_index = json.load(fp)
    index_word = {index:word for word,index in word_index.items()} 

### 2.3 - Clean, index and padd

In [102]:
max_len    = 256
vocab_size = 10000


nb_reviews = len(reviews)
x_data     = []

# ---- For all reviews
for review in reviews:
    # ---- First index must be <start>
    index_review=[1]
    # ---- For all words
    for w in review.split(' '):
        # ---- Clean it
        w_clean = re.sub(r"[^a-zA-Z0-9]", "", w)
        # ---- Not empty ?
        if len(w_clean)>0:
            # ---- Get the index
            w_index = word_index.get(w,2)
            if w_index>vocab_size : w_index=2
            # ---- Add the index if < vocab_size
            index_review.append(w_index)
    # ---- Add the indexed review
    x_data.append(index_review)    

# ---- Padding
x_data = keras.preprocessing.sequence.pad_sequences(x_data, value   = 0, padding = 'post', maxlen  = max_len)

### 2.4 - Have a look

In [91]:
def translate(x):
    return ' '.join( [index_word.get(i,'?') for i in x] )

for i in range(nb_reviews):
    imax=np.where(x_data[i]==0)[0][0]+5
    print(f'\nText review      :',    reviews[i])
    print(  f'x_train[{i:}]       :', list(x_data[i][:imax]), '(...)')
    print(  'Translation      :', translate(x_data[i][:imax]), '(...)')


Text review      : This film is particularly nice, a must see.
x_train[0]       : [1, 2, 22, 9, 572, 2, 6, 215, 2, 0, 0, 0, 0, 0] (...)
Translation      : <start> <unknown> film is particularly <unknown> a must <unknown> <pad> <pad> <pad> <pad> <pad> (...)

Text review      : Some films are classics and cannot be ignored.
x_train[1]       : [1, 2, 108, 26, 2239, 5, 566, 30, 2, 0, 0, 0, 0, 0] (...)
Translation      : <start> <unknown> films are classics and cannot be <unknown> <pad> <pad> <pad> <pad> <pad> (...)

Text review      : This movie is just abominable and doesn't deserve to be seen!
x_train[2]       : [1, 2, 20, 9, 43, 2, 5, 152, 1833, 8, 30, 2, 0, 0, 0, 0, 0] (...)
Translation      : <start> <unknown> movie is just <unknown> and doesn't deserve to be <unknown> <pad> <pad> <pad> <pad> <pad> (...)


## Step 2 - Bring back the model

In [83]:
model = keras.models.load_model('./run/models/best_model.h5')

## Step 4 - Predict

In [86]:
y_pred   = model.predict(x_data)

#### And the winner is :

In [89]:
for i in range(nb_reviews):
    print(f'\n{reviews[i]:<70} =>',('NEGATIVE' if y_pred[i][0]<0.5 else 'POSITIVE'),f'({y_pred[i][0]:.2f})')


This film is particularly nice, a must see.                            => POSITIVE (0.62)

Some films are classics and cannot be ignored.                         => POSITIVE (0.57)

This movie is just abominable and doesn't deserve to be seen!          => NEGATIVE (0.42)


In [101]:
a=[1]+[i for i in range(3)]
a

[1, 0, 1, 2]